# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

c:\Users\Balkan\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))


In [5]:
parquet_files

['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015.pa

In [6]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
len(parquet_files)

11192

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
# Write your code below.


import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low']
)))

C:\Users\Balkan\AppData\Local\Temp\ipykernel_26284\414211215.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [8]:
pdd_feat = dd_feat.compute()
pdd_feat



,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
MSI,2015-01-02,67.540001,67.730003,66.360001,66.510002,56.538261,1077900,Information Technology,Communications Equipment,2015,NaN,NaN,1.370003
MSI,2015-01-05,66.239998,66.470001,64.959999,65.059998,55.305649,1628800,Information Technology,Communications Equipment,2015,66.510002,-0.021801,1.510002
MSI,2015-01-06,65.000000,65.220001,64.110001,64.510002,54.838097,1738500,Information Technology,Communications Equipment,2015,65.059998,-0.008454,1.110001
MSI,2015-01-07,65.000000,65.000000,63.930000,64.430000,54.770100,1567600,Information Technology,Communications Equipment,2015,64.510002,-0.001240,1.070000
MSI,2015-01-08,64.910004,65.629997,64.879997,65.430000,55.620155,1234800,Information Technology,Communications Equipment,2015,64.430000,0.015521,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCG,2019-12-24,10.930000,11.070000,10.770000,10.950000,10.930913,4972000,Utilities,Multi-Utilities,2019,10.940000,0.000914,0.299999
PCG,2019-12-26,10.950000,11.000000,10.760000,10.860000,10.841070,6296300,Utilities,Multi-Utilities,2019,10.950000,-0.008219,0.240000
PCG,2019-12-27,10.670000,10.780000,10.310000,10.440000,10.421803,9534600,Utilities,Multi-Utilities,2019,10.860000,-0.038674,0.469999


In [9]:
pdd_feat.describe()

,Date,Open,High,Low,Close,Adj Close,Volume,year,Close_lag_1,returns,hi_lo_range
count,2736038,2.736038e+06,2.736038e+06,2.736038e+06,2.736038e+06,2.736038e+06,2.736038e+06,2.736038e+06,2.735541e+06,2.735541e+06,2.736038e+06
mean,2012-12-06 11:17:16.835600384,7.817649e+01,7.908428e+01,7.724908e+01,7.818894e+01,6.905217e+01,7.986788e+06,2.012435e+03,7.818370e+01,4.597581e-03,1.835200e+00
min,2000-01-03 00:00:00,3.020800e-02,3.052100e-02,2.697900e-02,3.052100e-02,3.052100e-02,0.000000e+00,2.000000e+03,3.052100e-02,-9.992177e-01,0.000000e+00
25%,2007-01-18 00:00:00,2.358000e+01,2.391000e+01,2.325000e+01,2.358000e+01,1.677972e+01,9.255000e+05,2.007000e+03,2.358000e+01,-9.171200e-03,4.780006e-01
50%,2013-04-11 00:00:00,4.380000e+01,4.431000e+01,4.327000e+01,4.381000e+01,3.360133e+01,2.100400e+06,2.013000e+03,4.380250e+01,4.856529e-04,8.999977e-01
75%,2018-12-28 00:00:00,8.145000e+01,8.231000e+01,8.058000e+01,8.147000e+01,7.013159e+01,4.973258e+06,2.018000e+03,8.147000e+01,1.019023e-02,1.779999e+00
max,2024-07-05 00:00:00,8.097980e+03,8.211400e+03,8.010000e+03,8.099960e+03,8.099960e+03,9.230856e+09,2.024000e+03,8.099960e+03,2.090455e+02,6.572100e+02
std,NaN,1.699010e+02,1.719153e+02,1.679210e+02,1.699476e+02,1.669752e+02,4.751373e+07,6.990235e+00,1.699460e+02,3.562026e-01,4.718255e+00


In [10]:
pdd_feat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2736038 entries, MSI to PCG
Data columns (total 13 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Date         datetime64[ns]
 1   Open         float64       
 2   High         float64       
 3   Low          float64       
 4   Close        float64       
 5   Adj Close    float64       
 6   Volume       int64         
 7   sector       object        
 8   subsector    object        
 9   year         int32         
 10  Close_lag_1  float64       
 11  returns      float64       
 12  hi_lo_range  float64       
dtypes: datetime64[ns](1), float64(8), int32(1), int64(1), object(2)
memory usage: 281.8+ MB


In [11]:
pdd_feat['rolling_avg_return'] = pdd_feat.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())
pdd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,
MSI,2015-01-02,67.540001,67.730003,66.360001,66.510002,56.538261,1077900,Information Technology,Communications Equipment,2015,NaN,NaN,1.370003,NaN
MSI,2015-01-05,66.239998,66.470001,64.959999,65.059998,55.305649,1628800,Information Technology,Communications Equipment,2015,66.510002,-0.021801,1.510002,NaN
MSI,2015-01-06,65.000000,65.220001,64.110001,64.510002,54.838097,1738500,Information Technology,Communications Equipment,2015,65.059998,-0.008454,1.110001,NaN
MSI,2015-01-07,65.000000,65.000000,63.930000,64.430000,54.770100,1567600,Information Technology,Communications Equipment,2015,64.510002,-0.001240,1.070000,NaN
MSI,2015-01-08,64.910004,65.629997,64.879997,65.430000,55.620155,1234800,Information Technology,Communications Equipment,2015,64.430000,0.015521,0.750000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCG,2019-12-24,10.930000,11.070000,10.770000,10.950000,10.930913,4972000,Utilities,Multi-Utilities,2019,10.940000,0.000914,0.299999,-0.009665
PCG,2019-12-26,10.950000,11.000000,10.760000,10.860000,10.841070,6296300,Utilities,Multi-Utilities,2019,10.950000,-0.008219,0.240000,-0.009026
PCG,2019-12-27,10.670000,10.780000,10.310000,10.440000,10.421803,9534600,Utilities,Multi-Utilities,2019,10.860000,-0.038674,0.469999,-0.009434


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No it was not necessary.
+ Would it have been better to do it in Dask? Why?
Yes, it is absolutely necessary to use Dask because it is faster and takes up less space.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.